In [ ]:
import utility 
import numpy as np
from importlib import reload
reload(utility)
import sys
import time
from tqdm import tqdm
import matplotlib.pyplot as pl

HOME = '/cluster/home/ajoudaki/projects2019-string-embedding/python_tools'

file_name = 'seqs9'
data_path = HOME + '/data/' + file_name + '.npz'
result_path = HOME + '/results/' + file_name 

Res = np.load(data_path)


seqs = Res['seqs']
vals = Res['vals']
options = Res['options']
Op = (options[()])
if int(file_name[4:])>=7:
    gene_lens = Res['gene_lens']
    num_seqs = Res['num_seqs']
else:
    num_seqs = [Op.num_seq]*Op.repeat

print('num seqs = ', len(seqs), ' mean lenth of seqs = ', np.mean([len(seqs[i]) for i in range(len(seqs))]))
print(Op)

In [11]:
proj_dim = 120
k_small = 2 
k_big = 100
num_trees = 2
num_neighbours = 200
step_build = 5
step_search = 5


kmers, s_kmer_vals, kmer_pos, kmer_seq_id = utility.list_kmers_simple(seqs, vals = vals,  
                                         k = k_small, addy = True, padding = int(k_big))
kmer_vals = utility.get_kmver_vals(s_kmer_vals, k_big)
build_indices = np.arange(0,len(kmers),step_build)
search_indices = np.arange(0,len(kmers),step_search)

convolved = utility.random_projection(kmers, k_big, proj_dim)

knn_index = utility.build_index(convolved, build_indices, num_trees)
NN,NN_dist = utility.knn_search_value(knn_index, convolved, search_indices, num_neighbours)
    
quad = utility.eval_results(search_indices, NN, kmers, kmer_vals, kmer_pos, kmer_seq_id, num_seqs, options)

print('seq len = ', len(seqs[0]), ', num seqs = ', len(seqs))

print('hit ratio: ', len(quad)/len(search_indices)/num_neighbours)

computing the list of kmers and values 


100%|██████████| 50/50 [00:00<00:00, 125.57it/s]


 computing the projection


100%|██████████| 559685/559685 [00:00<00:00, 801643.33it/s]

 computing the convolution


...

time to build 2 trees = 2.300187587738037
nearest neighbours search:


100%|██████████| 111937/111937 [00:21<00:00, 5324.80it/s]

time search = 21.025116443634033



100%|██████████| 111937/111937 [00:17<00:00, 6288.48it/s]

evaluation time =  17.813982725143433
folls possitive =  0.9981407308584594
recall percentage  0.6871111111111111
seq len =  11000 , num seqs =  50
hit ratio:  0.001764385323887544


In [3]:
# print(Op)